In [30]:
import pandas as pd
import requests
import random
import time
from time import sleep
from bs4 import BeautifulSoup
from random import randint
def Check_number_of_pages(link,n):
    # Parse HTML text
    
    for x in range(1,n):
        r = requests.get(f"{link}&page={x}")
        soup = BeautifulSoup(r.text, 'html.parser')
        products_check = soup.find_all('div',{'class':'product-item'})
        items = len(products_check)
        y = x
        sleep(random.randint(1,1)) 
        if items == 0:
            pages = y-1
            print(f'{pages} is the final pages of this product')
            return pages 
        else:
            print(f'{y-1} is not the final pages of this product')
    return pages

    
def crawling_product(url1,number_of_pages,pdn):
    """Scrape the page of product you want to
      Input: url to the webpage. Default: https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner
      Output: A list containing scraped data of all product & price
    """
    # Get parsed HTML
    dataall = []   
    x = 1
    while x <= number_of_pages:
        url=f"{url1}&page={x}"
        r = requests.get(url)
        # Parse HTML text
        soup = BeautifulSoup(r.text, 'html.parser')  
        # Find all products in the page
        products = soup.find_all('div',{'class':'product-item'}) 
        x += 1
        data = []
        # List containing data of all products 
        for product in products: 
            d = {'Product-id':'','Seller-id':'','Product-SKU':'','Product-Title':'','Product-Brand':'',
                 'URL':'','img_URL':'','Regular-Price':'','Final-Price':'','Author':'','Comment':'','TIKI-NOW':'','Rating':'','Installment':'',
                 'Page':'','Location':''}
            #other optional : ,'Rating':''
            try:
                d['Product-id'] = product.a['data-id']
                d['Product-SKU'] = product['product-sku']
                d['Seller-id'] = product['data-seller-product-id']
                d['Product-Title'] = product.a['title']
                d['Product-Brand'] = product['data-brand']
                
                d['URL'] = (f"https://tiki.vn{product.a['href']}")
                d['img_URL'] = product.a.div.span.img['src']
                Finalprice = product.find('span',{'class':'final-price'})
                d['Final-Price'] = Finalprice.text.strip(" \n, ...")
                d['Location'] = products.index(product)
                d['Page'] = x-1
                
#Let's Check for somethings deeper
                
                try:
                    Author = product.find('p',{'class':'author'})
                    d['Author'] = Author.text
                except:
                    d['Author'] = None
                #COMMENT
                try: 
                    Comment = product.find('p',{'class':'review'})
                    d['Comment'] = Comment.text.strip(" \n, ')' (... ")
                except:
                    print(f"No comment for {d['Product-Title']}")
                #REGULAR PRICE
                try:
                    #Some products doesn't have regular price
                    Regularprice = product.find('span',{'class':'price-regular'})
                    d['Regular-Price'] = Regularprice.text.strip(" \n, ...")
                except:
                    print(f"No Regular Price for {d['Product-Title']}")    
                    d['Regular-Price'] = d['Final-Price']
                #INSTALLMENT
                try:                          
                    #Or somethings like Installment
                    Installment = product.find('span',{'class':'installment-price-v2'})
                    d['Installment'] = Installment.text
                except:
                    print(f"No installment for {d['Product-Title']}")
                    d['Installment'] = None                

                    #rating is somes of optional
                try:
                    Rating = product.find('span',{'class':'rating-content'})
                    d['Rating'] = str(Rating.span['style']).strip("width:")
                except:
                    print(f"No Rating for {d['Product-Title']}")
                    d['Rating'] = None                       

                try:
                    #TIKI-NOW - aka Unique Value of TIKI
                    tikinow = product.find('div',{'class':'badge-service'})
                    badge_check = tikinow.div.img['src']
                    if str(badge_check) == "https://salt.tikicdn.com/ts/upload/9f/32/dd/8a8d39d4453399569dfb3e80fe01de75.png":
                        d['TIKI-NOW'] = "YES"
                except:
                    d['TIKI-NOW'] = "NO"
                    
                #Then we appending all the Data from this page:       
                data.append(d)   
            #If missing some info let's the syntax warning back.
            except:
                print("We got one product missing somes info!")
        #Then let's bring all data of products from one page to extend one data name "dataall"   
        dataall.extend(data)
        #Use pandas to DF the dataall, with colums keys is"dataall[0]""     
        full_products = pd.DataFrame(data = dataall, columns = dataall[0].keys())
        #Then convert it to CSV for saving & visualization and name it by "pdn" = product-name
        full_products.to_csv(f'./Full_Products_{pdn}.csv', index=False)
           
    return full_products
    sleeptime = random.randint(1,2)
    sleep(sleeptime)

# link ='https://tiki.vn/cham-soc-da-mat/c1582?src=tree&_lc=Vk4wMzkwMjMwMDg%3D&src=c.1582.hamburger_menu_fly_out_banner'
# crawling_product(link, Check_number_of_pages(link,60), 'SkinCare')

In [33]:
catalogue = input(f"What's Products/Catalogues you want to crawl: ")
link = input(f"Instert the Url of the product(first page) :")
n = 99999
crawling_product(link, Check_number_of_pages(link,n),catalogue)

What's Products/Catalogues you want to crawl: Laptops
Instert the Url of the product(first page) :https://tiki.vn/laptop-may-vi-tinh-linh-kien/c1846?src=c.1846.hamburger_menu_fly_out_banner
0 is not the final pages of this product
1 is not the final pages of this product
2 is not the final pages of this product
3 is not the final pages of this product
4 is not the final pages of this product
5 is not the final pages of this product
6 is not the final pages of this product
7 is not the final pages of this product
8 is not the final pages of this product
9 is not the final pages of this product
10 is not the final pages of this product
11 is not the final pages of this product
12 is not the final pages of this product
13 is not the final pages of this product
14 is not the final pages of this product
15 is not the final pages of this product
16 is not the final pages of this product
17 is not the final pages of this product
18 is not the final pages of this product
19 is not the final pa

No installment for RAM Laptop Samsung 8GB DDR3L bus 1600 - Hàng Nhập Khẩu
No installment for Ổ Cứng SSD Lexar NS100 2.5 inch SATA III - Hàng Nhập Khẩu
No installment for Màn Hình Samsung 24' LS24E45KDSC/XV (1920x1080/TN/60Hz/5ms) - Hàng Chính Hãng
No installment for Bộ thu phát không dây Usb bluetooth 5.0 dùng cho máy tính laptop  D558
No installment for Ổ Cứng Di Động Gắn Ngoài SSD Sandisk Extreme Portable 500GB - Hàng Nhập Khẩu
No installment for Ổ Cứng HDD WD Blue™ 1TB/64MB/7200rpm/3.5 - WD10EZEX - Hàng chính hãng
No installment for Chuột Không Dây Logitech M275 - Hàng Chính Hãng 
No installment for Hộp đựng ổ cứng 2,5' SSD/HDD SATA 3 Orico 2577U3(HDD Box 2,5' USB 3.0) - Hàng Nhập Khẩu
No installment for Bộ vi xử lý - CPU Intel Core i3-9100F Processor (6M Cache, up to 4.20 GHz)- Hàng Chĩnh Hãng
No installment for Bàn Phím Không Dây Logitech K480 - Hàng Chính Hãng
No installment for Bàn phím không dây bluetooth cho iphone, ipad, samsung, smartphone (trắng)
No installment for Chuột Có

No installment for Màn Hình Asus VZ229HE 22inch FullHD 5ms 76Hz IPS - Hàng Chính Hãng
No installment for USB SanDisk CZ600 32GB - USB 3.0 - Hàng Chính Hãng
No installment for Bộ Kích Sóng Wifi Repeater Băng Tần Kép AC750 TP-Link  RE200 - Hàng Chính Hãng
No installment for USB SanDisk 3.0 Ultra CZ48 32GB - Hàng Chính Hãng
No installment for Router Wifi Chuẩn N TP-Link TL-WR840N - Hàng Chính Hãng
No installment for USB Bluetooth 4.0 dùng cho máy tính Laptop, PC
No installment for Bo Mạch Chủ Mainboard ASRock B450M Pro4-F Socket AM4 - Hàng Chính Hãng
No installment for Bàn phím bluetooth Mini KB - A6	  
No installment for USB Kingston DT100G3 128GB chính hãng
No installment for USB 3.0 SanDisk Ultra Flair CZ73 64GB 150MB/s chính hãng
No installment for USB OTG SanDisk Ultra Dual 32GB Type-C USB 3.1 DDC2 SDDDC2-032G-G46 - Hàng Chính Hãng
No installment for Ổ Cứng SSD Samsung 860 Qvo 1TB Sata III 2.5 inch - Hàng Nhập Khẩu (Box Tiếng Anh)
No installment for Đế tản nhiệt laptop 2 quạt N99 – c

No installment for Caddy bay chuẩn SATA 9.5mm 12.7mm nhôm sần cao cấp, tặng tovit
No installment for Router Gigabit Wi-Fi Băng Tần Kép AC1750 TP-Link Archer C7 - Hàng Chính Hãng
No installment for Bộ thu Wifi USB LB - LINK WN151 - Hàng chính hãng
No installment for Ổ Cứng Di Động SSD Samsung T7 NON Touch USB Type C - Hàng Nhập Khẩu - 500GB
No Regular Price for Đầu bấm mạng RJ45 loại xịn AMP (Hộp 100 hạt)
No installment for Đầu bấm mạng RJ45 loại xịn AMP (Hộp 100 hạt)
No installment for Ổ Cứng SSD Samsung M871A 128gb 2.5-inch sata iii - Hàng Nhập Khẩu
No installment for Chuột Không Dây Fuhlen A09G/A09B - Hàng Chính Hãng 
No Rating for Laptop Asus VivoBook S15 S533FA-BQ011T (Core i5-10210U/ 8GB RAM/ 512GB SSD/ 15.6 FHD/ Win10) - Hàng Chính Hãng
No installment for Bộ Phát WiFi Di Động Từ Sim 3G/4G Zte MF910 Tốc Độ 42Mbps, Pin 2300mAh (Hàng Chính Hãng)
No installment for CÁP MẠNG LAN BẤM SẴN 2 ĐẦU DÀI 20M
No installment for Bộ nhớ RAM máy tính CORSAIR Vengeance RGB Pro CMW16GX4M2E3200C16 

No installment for USB 3.0 SanDisk Ultra Flair CZ73 16GB - Hàng Nhập Khẩu
No installment for USB Sound Bluetooth
No installment for Ổ Cứng HP USB
No installment for Thiết Bị Phát Wifi Tenda AC6 Chuẩn AC1200 - Hàng Nhập Khẩu
No Rating for Laptop Asus Vivobook D409DA-EK151T (AMD Ryzen R3-3200U/ 4GB DDR4/ 256GB PCIE/ 14 FHD/ Win10) - Hàng Chính Hãng
No installment for 10 cuộn Giấy decal nhiệt, in tem mã vạch 2 tem, 35x22mm, dài 30m
No Rating for Laptop ASUS ROG Strix G15 G512L-VAZ068T (Core i7-10750H/ 16GB (8GBx2) DDR4 3200MHz/ 512GB SSD PCIE G3X4/ RTX 2060 6GB GDDR6/ 15.6 FHD IPS, 240Hz, 3ms/ Win10) - Hàng Chính Hãng
No installment for D-Link DIR-612 - Bộ phát Wifi chuẩn N 300Mbps Mở rộng sóng - Hàng Chính Hãng
No installment for Bộ Phát WiFi Từ Sim 3G/4G ZTE Mf65 Tốc Độ Cao (Hàng Chính Hãng)
No installment for Bộ kích sóng wifi 3 râu
No installment for Chuột không dây Genius NX7015 Vàng - Hàng Chính Hãng
No installment for Chuột LDK.ai D1
No installment for Quạt Tản Nhiệt 9x9x2.5Cm 12VD

No installment for Dây Cáp Mạng Orico PUG-C6-200 CAT6 20m - Hàng Chính Hãng
No installment for Ổ Cứng SSD Samsung mSATA PM851 128GB - Hàng Nhập Khẩu
No installment for Ổ Cứng HDD NAS WD Red 4TB/256MB/5400/3.5 SATA III - WD40EFAX - Hàng Chính Hãng
No Rating for Laptop HP ENVY 13-ba0046TU (Intel core i5-1035G4, RAM 8GB, 512GB SSD, 13.3'FHD, Win10 Home 64,Office,Gold_171M7PA - Hàng Chính Hãng
No installment for Dây Cáp Mạng Ugreen CAT7 11268 (1m) - Hàng Chính Hãng
No installment for Pin Bộ Phát Wifi 4G TP-Link M7200 - Hàng chính hãng
No installment for Sạc Dành Cho Macbook Air 14.85V-3.05A - 45W, Magsafe 2,Model A1436, A1465, Năm 2012 đến 2017
No installment for MÁY CHIẾU MINI FULL HD YG520
No installment for Keo tản nhiệt CPU Cooler Master Mastergell Pro công nghệ NANO - Hãng chính hãng
No Rating for Laptop Asus ROG Zephyrus Duo 15 GX550LWS-HF102T (Core i7-10875H/ 16GB DDR4 3200MHz/ 1TB SSD PCIE G3X4/ RTX 2070 Super 8GB GDDR6 - Max-Q/ 15.6 FHD IPS, 300Hz G-Sync, 3ms/ Win10) - Hàng Chính 

No installment for RAM GSKILL 8GB F4 2666C19S-8GIS DDR4 - Hàng Chính Hãng
No installment for Máy In Phun Màu Đa Năng Canon PIXMA E560 Scan Copy Chính Hãng Lê Bảo Minh
No installment for RAM DDR4 G.Skill 8GB (2800) F4-2800C17S-8GVR - Hàng Chính Hãng
No installment for Đế tản nhiệt laptop N99 2 quạt đế nâng - Hàng nhập khẩu
No installment for RAM PC DDR4 Samsung 8GB Bus 2666 - Hàng Nhập Khẩu
No installment for Ổ Cứng SSD WD Blue 3D NAND 250GB M.2 2280 WDS250G2B0B - Hàng Chính Hãng
No installment for Ram PC Kingston HyperX Fury Black 16GB (2x8GB) Bus 2666 DDR4 CL16 DIMM XMP Non-ECC HX426C16FB3K2/16 - Hàng Chính Hãng
No installment for Màn Hình HP V194 19inch HD 5ms 60Hz TN - Hàng Chính Hãng
No installment for Máy In Hóa Đơn Xprinter XP-58iiH - Hàng Nhập Khẩu
No installment for Laptop Dell Vostro 5590 70197465 (Core i5-10210U/ 8GB DDR4 2666MHz/ 256GB PCIe NVMe/ 15.6 FHD/ Win10) - Hàng Chính Hãng
No installment for Card Màn hình Asus PH GTX 1650 4G - Hàng chính hãng 
No Rating for Microsoft

No installment for Dell Optiplex  Core i3-3220, Ram 8gb, SSD 120GB - Hàng Nhập Khẩu
No installment for Vỏ case máy tính Aerocool BOLT MINI ACRYLIC - Hàng Chính Hãng
No installment for Giấy in nhiệt, in mã vạch, in dán tem phụ, in tem cân 58x40mm
No Rating for Laptop ASUS TUF Gaming A15 FA506IU-AL127T (AMD Ryzen 7 4800H/ 8GB DDR4 3200MHz/ 512GB SSD M.2 PCIE G3X2/ GTX 1660Ti 6GB GDDR6/ 15.6 FHD IPS, 144Hz/ Win10) - Hàng Chính Hãng
No installment for Ổ Cứng Di Dộng HDD WD My Passport 2TB 2.5' USB 3.2 WDBYVG0020B - Hàng Chính Hãng
No installment for Bộ 4 Màu Mực in phun Thuận Phong TP50 (100ml) dùng cho máy in phun Epson, HP, Canon - Hàng Chính Hãng
No Regular Price for Bàn Phím Dành Cho Laptop Dell Inspiron 3441 3442 3443 3446 3451 3458 - Hàng Nhập Khẩu
No installment for Bàn Phím Dành Cho Laptop Dell Inspiron 3441 3442 3443 3446 3451 3458 - Hàng Nhập Khẩu
No installment for Máy chấm công vân tay A6
No installment for Bộ Phát Sóng Wifi 300Mbps 2 Ăng-Ten TPLink 820N- Hàng Chính Hãng
No Rat

,Product-id,Seller-id,Product-SKU,Product-Title,Product-Brand,URL,img_URL,Regular-Price,Final-Price,Author,Comment,TIKI-NOW,Rating,Installment,Page,Location
0,3054369,3054371,5116522285995,Phần Mềm Diệt Virus BKAV Profressional 1 PC 12...,Bkav,https://tiki.vn/phan-mem-diet-virus-bkav-profr...,https://salt.tikicdn.com/cache/280x280/ts/prod...,299.000đ,194.000đ ...,None,710 nhận xét,YES,98%,None,1,0
1,547563,271577,7506266142100,Bộ Kích Sóng Wifi Repeater 300Mbps Totolink EX...,TOTOLINK,https://tiki.vn/bo-kich-song-wifi-repeater-300...,https://salt.tikicdn.com/cache/280x280/media/c...,273.000đ,199.000đ ...,None,1351 nhận xét,YES,82%,None,1,1
2,416613,119779,7505256458276,TP-Link TL-WR841N - Router Wifi Chuẩn N Tốc Đ...,TP-Link,https://tiki.vn/tp-link-tl-wr841n-router-wifi-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,359.000đ,282.000đ ...,None,477 nhận xét,YES,88%,None,1,2
3,405243,107557,6105007657623,USB Kingston DT100G3 32GB USB 3.0 - Hàng Chính...,Kingston,https://tiki.vn/usb-kingston-dt100g3-32gb-usb-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,365.000đ,91.000đ ...,None,1303 nhận xét,YES,86%,None,1,3
4,356188,49932,6703667868931,Chuột Có Dây Logitech B100 - Hàng Chính Hãng,Logitech,https://tiki.vn/chuot-co-day-logitech-b100-han...,https://salt.tikicdn.com/cache/280x280/ts/prod...,90.000đ,65.000đ ...,None,1045 nhận xét,YES,92%,None,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,514911,1030588,7502794336646,TP-Link Archer T6E - Bộ Chuyển Đổi PCI Expres...,TP-Link,https://tiki.vn/tp-link-archer-t6e-bo-chuyen-d...,https://salt.tikicdn.com/cache/280x280/media/c...,999.000đ,845.000đ ...,None,23 nhận xét,YES,74%,None,21,35
1000,49206793,51090770,3519305181286,Microsoft Surface Laptop 3 (Core i5-1035G7/13'...,Microsoft,https://tiki.vn/microsoft-surface-laptop-3-cor...,https://salt.tikicdn.com/cache/280x280/ts/prod...,29.990.000đ,23.480.000đ ...,None,2 nhận xét,YES,80%,None,21,36
1001,24727789,24727790,7758949144677,Giấy In Nhãn Dán (Decal) Media Mặt Mờ A4 100 T...,Media Ink,https://tiki.vn/giay-in-nhan-dan-decal-media-m...,https://salt.tikicdn.com/cache/280x280/ts/prod...,119.000đ,82.000đ ...,None,8 nhận xét,YES,90%,None,21,37
1002,13728873,13728874,4064857878775,Bo Mạch Chủ Mainboard ASRock B365M Pro4 Socket...,ASRock,https://tiki.vn/bo-mach-chu-mainboard-asrock-b...,https://salt.tikicdn.com/cache/280x280/ts/prod...,2.269.000đ,1.789.000đ ...,None,2 nhận xét,YES,90%,None,21,38


In [8]:
#FOR TESTING
import pandas as pd
import requests
import random
import time
from time import sleep
from bs4 import BeautifulSoup
from random import randint
url="https://tiki.vn/bo-trang-diem/c11813?src=c.1584.hamburger_menu_fly_out_banner&_lc=Vk4wMzkwMjMwMDg%3D"
r = requests.get(url)
# Parse HTML text
soup = BeautifulSoup(r.text, 'html.parser')  
# Find all products in the page
products = soup.find_all('div',{'class':'product-item'}) 
data = []
# List containing data of all products 
for product in products: 
    d = {'Product-id':'','Seller-id':'','Product-SKU':'','Product-Title':'','Product-Brand':'',
         'URL':'','img_URL':'','Regular-Price':'','Final-Price':'','Comment':'','TIKI-NOW':'','Installment':'','Location':''}
    #other optional : ,'Rating':'',,'Comment':'''Rating':'',
    try:
        d['Product-id'] = product.a['data-id']
        d['Product-SKU'] = product['product-sku']
        d['Seller-id'] = product['data-seller-product-id']
        d['Product-Title'] = product.a['title']
        d['Product-Brand'] = product['data-brand']
        d['URL'] = (f"https://tiki.vn{product.a['href']}")
        d['img_URL'] = product.a.div.span.img['src']
        Finalprice = product.find('span',{'class':'final-price'})
        d['Final-Price'] = Finalprice.text.strip(" \n, ...")
        d['Location'] = products.index(product)
        try:
            Comment = product.find('p',{'class':'review'})
            d['Comment'] = Comment.text.strip(" \n, ')' (... ")
            #d['Page'] = x-1
            #Let's Check for somethings deeper
            #Some products doesn't have regular price
            Regularprice = product.find('span',{'class':'price-regular'})
            d['Regular-Price'] = Regularprice.text.strip(" \n, ...")
            #Or somethings like Installment
            Installments = product.find('span',{'class':'installment-price-v2'})
            d['Installment'] = Installments.text
            #rating is somes of optional
            #Rating = product.find('span',{'class':'rating-content'})
            #d['Rating'] = Rating['style']
            #TIKI-NOW - aka Unique Value of TIKI
            tikinow = product.find('div',{'class':'badge-service'})
            badge_check = tikinow.div.img['src']
            if str(badge_check) == "https://salt.tikicdn.com/ts/upload/9f/32/dd/8a8d39d4453399569dfb3e80fe01de75.png":
                d['TIKI-NOW'] = "YES"
            else:
                d['TIKI-NOW'] = "NO"
        
        except:
            #d['Regular-Price'] = d['Final-Price']
            d['Installment'] = None
            
            #d['Rating'] = Non              
    except:
        
        #If missing some info let's the syntax warning back.
        print("We got one product missing somes info!")
         #Then we appending all the Data from this page:
    data.append(d)
finaaa = pd.DataFrame(data = data, columns = data[0].keys())
finaaa
#Then let's bring all data of products from one page to extend one data name "dataall" 

,Product-id,Seller-id,Product-SKU,Product-Title,Product-Brand,URL,img_URL,Regular-Price,Final-Price,Comment,TIKI-NOW,Installment,Location
0,40972439,44066357,5840306531694,BẢNG PHẤN MẮT 35 MÀU COCO VENUS hãng DIKALU bả...,DIKALU,https://tiki.vn/bang-phan-mat-35-mau-coco-venu...,https://salt.tikicdn.com/cache/280x280/ts/prod...,170.500đ,150.000đ ...,59 nhận xét,,None,0
1,52355165,52355166,2963814419452,Bộ Trang Điểm Kiss Beauty Đỏ 5 Sản Phẩm Touch ...,Kiss Beauty,https://tiki.vn/bo-trang-diem-kiss-beauty-do-5...,https://salt.tikicdn.com/cache/280x280/ts/prod...,185.000đ,99.000đ ...,3 nhận xét,,None,1
2,3896043,55291205,8776343211770,Bộ Mỹ Phẩm Trang Điểm Hàn Quốc 01 The Rucy PPM211,The Rucy,https://tiki.vn/bo-my-pham-trang-diem-han-quoc...,https://salt.tikicdn.com/cache/280x280/ts/prod...,589.000đ,299.000đ ...,17 nhận xét,,None,2
3,39999755,39999761,7893390802971,Combo Chì Kẻ Chân Mày#1 Và Phấn Phủ Kiềm Dầu I...,I AM SODA,https://tiki.vn/combo-chi-ke-chan-may-1-va-pha...,https://salt.tikicdn.com/cache/280x280/ts/prod...,380.000đ,99.000đ ...,12 nhận xét,,None,3
4,59405609,59405610,2573740685793,Bộ Trang Điểm Mắt Chuẩn Salon (Mascara Falsie ...,Maybelline NewYork,https://tiki.vn/bo-trang-diem-mat-chuan-salon-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,464.000đ,348.000đ ...,2 nhận xét,,None,4
5,7884757,7884758,3408926744775,"Combo mút trang điểm hình nấm tán kem nền, phấ...",Minigood,https://tiki.vn/combo-mut-trang-diem-hinh-nam-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,149.000đ,109.000đ ...,3 nhận xét,,None,5
6,56589598,56589600,1556101373431,Combo Son kem lì Lemonade Perfect Couple Lip v...,Lemonade,https://tiki.vn/combo-son-kem-li-lemonade-perf...,https://salt.tikicdn.com/cache/280x280/ts/prod...,488.000đ,348.000đ ...,Chưa có nhận xét,,None,6
7,3896039,4125429,6243580461159,Bộ Trang Điểm Love My Skin 04 The Rucy PNSM1,The Rucy,https://tiki.vn/bo-trang-diem-love-my-skin-04-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,768.000đ,250.000đ ...,4 nhận xét,,None,7
8,66643543,66643544,3419674175186,[Bộ trang điểm] Kem nền kiềm dầu chống nắng Fi...,Maybelline NewYork,https://tiki.vn/bo-trang-diem-kem-nen-kiem-dau...,https://salt.tikicdn.com/cache/280x280/ts/prod...,526.000đ,299.000đ ...,Chưa có nhận xét,,None,8
9,26038122,26038125,8513182074568,Combo Mini Trang Điểm Và Dưỡng Da Ẩm Mượt Rạng...,Laneige,https://tiki.vn/combo-mini-trang-diem-va-duong...,https://salt.tikicdn.com/cache/280x280/ts/prod...,600.000đ,510.000đ ...,4 nhận xét,,None,9
